In [1]:
#basic imports
import numpy as np
import pandas as pd

#visualization libraries
import cufflinks as cf
cf.go_offline()

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.io as pio
from plotly.offline import iplot

import plotly.io as pio

In [2]:
exercises = pd.read_csv('exercises.csv')

In [3]:
volume = exercises.groupby('date')[['volume']].sum()

In [4]:
sleep = pd.read_csv('fitbit/sleep.csv')

In [5]:
sleep.duration = sleep.duration.div(1000).div(3600)

In [6]:
sleep = sleep.groupby('dateOfSleep', as_index = False).duration.sum()

In [7]:
sleep.dateOfSleep = pd.to_datetime(sleep.dateOfSleep) + pd.Timedelta('1 day')

In [8]:
sleep.set_index('dateOfSleep', inplace = True)

In [9]:
test = volume.join(sleep).query('duration > 0')

In [10]:
test.index = pd.to_datetime(test.index)

In [11]:
subplots = tls.make_subplots(rows=2, cols=2, 
                              specs=[[{}, {}], 
                                     [{}, {}]
                                    ],
                                 subplot_titles = ['Daily', 'Weekly', 'Monthly', 'Yearly'],
                              print_grid=True)

f = go.FigureWidget(subplots)

samples = ['D', 'W', 'M', 'A']

rows = [1, 1, 2, 2]
cols = [1, 2, 1, 2]

for sample, row, col in zip(samples, rows, cols):
    sampled = test.resample(sample).sum().query('volume > 0')
    f.add_scatter(x = sampled.duration, 
                  y = sampled.volume, 
                  mode = 'markers', 
                  marker = {'opacity': .5, 
                            'color': '#A61B0F',
                            'line': {'color': 'black', 'width': .5}
                           },
                  showlegend = False, 
                  row = row, 
                  col = col)

f.layout.title = '<b>Less Sleep Means Less Weight Lifted on a Weekly and Monthly Basis</b><br><i>Total Sleep vs. Total Volume Lifted'

f.layout.yaxis.title = 'Volume Lifted (lbs)'

f.layout.xaxis3.title = 'Hours of Sleep the Previous Night'

# set colors
f.layout.plot_bgcolor = '#E5E5E5'
f.layout.paper_bgcolor = '#E5E5E5'

f.layout.images = [dict(
        source="https://raw.githubusercontent.com/WilliamGreenlaw/Logos/master/signature_min.png",
        xref="paper", yref="paper",
        x=1.05, y=1.10,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [12]:
f

FigureWidget({
    'data': [{'marker': {'color': '#A61B0F', 'line': {'color': 'black', 'width': 0.5}, 'opacity…

In [13]:
pio.write_image(f, 'sleep.svg', width = 1500, height = 500)
pio.write_image(f, 'sleep.png', width = 1500, height = 500)